In [1]:
# the point of this notebook is to take ciliary beat frequency videos from KO cultures (no reference images),
# process them, and concatenate the resulting csvs into something that can be analyzed in r.

# requires:
# - beat frequency videos (256x256 .dvs with framerate = 8 ms)
# - csv key to points (culturepoint, vidpoint.start, vidpoint.end, donor, condition)
# optional:
# - reference images (1024 x 1024 (or 256x256...) .dv files, 4 channel)

# because the name scheme of every experiment was different, it is very important to check that everything is matched correctly
# refs and vids
# and then vidpoints to culturepoints.
# I had to manually edit concatenate_csvs for most experiments.
# The specific lines that probably need editing are commented in all caps.

In [2]:
# packages
import os # navigating file system
import mrc # opening .dv files
from natsort import natsorted # dealing with file pairs where one is named _01.dv and the other is _1.dv.
import numpy as np # numbers
import scipy # signal processing
from skimage import io as skio # for saving tif
import csv # writing csv
import datetime
import sys
import subprocess
import pandas as pd # for dataframe handling

# define functions

# write a log file so i can have some info about how the function was run
# ideally i would also include the specific code that was run however that seems to be pretty hard !
def generate_log(log_file_path):
    # get current date and time
    current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # get package versions
    try:
        package_versions = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).decode('utf-8')
    except subprocess.CalledProcessError:
        package_versions = "Failed to retrieve package versions."

    # create the log content
    log_content = f"Date: {current_date}\n\n"
    log_content += f"Code from ciliako_processing\n\n"
    log_content += "Package versions:\n"
    log_content += package_versions

    # Write log content to the log file
    with open(log_file_path, 'w') as log_file:
        log_file.write(log_content)



# look at files in a folder and find .dvs with ref in the appropriate position + matched non ref
# This particular iteration can handle refs or vids in tif or dv format.
def organize(folder):
    list = natsorted(os.listdir(folder))
    cilialist = []
    reflist = []
    
    for m in list:
        if (m.endswith('.dv') or m.endswith('.tif')) and 'ref' not in m:
            cilialist.append(m)
        elif (m.endswith('.dv') or m.endswith('.tif')) and 'ref' in m:
            reflist.append(m)
    return cilialist, reflist

# calculate power and dominant frequency for each pixel
def powerfreq_all(pix):
    f, pxx = scipy.signal.periodogram(pix, fs=125)
    max_pxx = np.max(pxx)
    index = np.where(pxx == max_pxx)
    domfreq = f[index]
    return domfreq[0], max_pxx


# the big function. works for videos with and without reference images.
def perpair_processing(folder):
    # Create processed folder if it doesn't exist
    processed_folder = os.path.join(folder, 'processed')
    os.makedirs(processed_folder, exist_ok=True)

    # Get lists of all the ciliavids and ref images in the folder
    ciliavids, refimgs = organize(folder)

    for x in range(len(ciliavids)):
        current = ciliavids[x]
        if x < len(refimgs):
            refname = refimgs[x]
        else:
            refname = None
        print('Reference: ' + str(refname))
        print('Video: ' + current)

        # Read cilia video
        if current.endswith('.dv'):
            # Read DV 
            image = mrc.imread(os.path.join(folder, current))
        elif current.endswith('.tif'):
            # Read TIFF image using skimage
            image = skio.imread(os.path.join(folder, current))
        rows, cols = image.shape[1:]
        
        # Read matched reference image
        if refname:
            if refname.endswith('.dv'):
                refimage = mrc.imread(os.path.join(folder, refname))
            elif refimage.endswith('.tif'):
                refimage = skio.imread(os.path.join(folder, refname))
            refproj = refimage.max(axis=1)
            if refproj.shape[1] > 256: #there was an incident where some reference iamges were not 1024 x 1024
                refcent = refproj[:, 383:639, 383:639]
            else:
                refcent = refproj
        else:
            refcent = np.empty((4, rows, cols))
            refcent.fill(np.nan)

        # Create empty arrays for storing pixel values
        power = np.empty((rows, cols))
        domfreq = np.empty((rows, cols))
        sumproj = np.empty((rows, cols))
        refintensity = np.empty((4, rows, cols))

        # Process each pixel
        for i in range(rows):
            for j in range(cols):
                domfreq[i, j], power[i, j] = powerfreq_all(image[:, i, j])
                sumproj[i, j] = image[:, i, j].sum()
                refintensity[:, i, j] = refcent[:, i, j]

        # Save power, dominant frequency, sum projection, and reference image intensity as TIFF
        output_tiff = np.concatenate((power[np.newaxis, :, :], domfreq[np.newaxis, :, :],
                                      sumproj[np.newaxis, :, :], refintensity), axis=0)
        tiff_filename = os.path.join(processed_folder, current[:-4] + '_nugget.tif')
        skio.imsave(tiff_filename, output_tiff, check_contrast=False)
        print('Saved: ' + tiff_filename)

        # Save pixel values as CSV
        output_csv = np.column_stack((np.repeat(current, rows*cols),
                                      np.repeat(refname, rows*cols),
                                      np.repeat(np.arange(rows), cols),
                                      np.tile(np.arange(cols), rows),
                                      power.ravel(), domfreq.ravel(), sumproj.ravel(),
                                      refintensity.reshape(4, -1).T))
        csv_filename = os.path.join(processed_folder, current[:-4] + '_perpixsummary.csv')
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['video', 'refimage', 'i', 'j', 'power', 'domfreq', 'sumproj',
                             'spytub', 'nucview', 'gfp', 'pol'])
            writer.writerows(output_csv)
        print('Saved: ' + csv_filename)
    logfilepath = folder + 'processed/log.txt'
    generate_log(logfilepath)
    return

In [3]:
### concatenate the csvs and add extra info ###

def concatenate_csvs(folder, mapping_csv, timepoints_csv):
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]

    # Read the mapping CSV file
    mapping_df = pd.read_csv(mapping_csv)
    timepoints_df = pd.read_csv(timepoints_csv)
    
    # Create an empty list to store the concatenated dataframes
    dfs = []

    # Iterate over the CSV files
    for file in csv_files:
        # Read each CSV file
        csv_path = os.path.join(folder, file)
        df = pd.read_csv(csv_path)

        # Extract the file information from the filename. Owing to the nature of image acquisition,
        # this will probably need adjustment for every experiment.
        filename = os.path.splitext(file)[0]
        timepoint = int(filename.split('_')[3][1:]) # CHANGE THIS FOR YOUR EXPERIMENT OF INTEREST
        point = int(filename.split('_')[4]) # CHANGE THIS FOR YOUR EXPERIMENT OF INTEREST

        # Lookup donor and mock/infected information from the mapping CSV
        # first, find the row of the mapping_df that has the correct info
        mask = (mapping_df['vidpoint.start'] <= point) & (mapping_df['vidpoint.end'] >= point)
        # then, take the info we need from that row
        donor = int(mapping_df.loc[mask, 'donor'].values[0])
        condition = mapping_df.loc[mask, 'condition'].values[0]
        culturepoint = int(mapping_df.loc[mask, 'culturepoint'].values[0])
        experiment = mapping_df.loc[mask, 'experiment'].values[0]
        
        # Lookup hpi from other mapping csv
        hpi = timepoints_df.loc[timepoints_df['timepoints'] == timepoint, 'hpi'].values[0]
        
        # Add the additional columns
        df['donor'] = donor
        df['vidpoint'] = point
        df['condition'] = condition
        df['culturepoint'] = culturepoint
        df['experiment'] = experiment
        df['timepoint'] = timepoint
        df['hpi'] = hpi
        
        # Append the dataframe to the list
        dfs.append(df)
        print(f'appended file {filename} donor {donor} {condition} pt# {culturepoint} time {hpi}')

    # Concatenate the dataframes
    result_df = pd.concat(dfs)

    # Save the concatenated dataframe to a new CSV file
    output_csv = os.path.join(folder, 'concatenated.csv')
    result_df.to_csv(output_csv, index=False)
    
    ## Make a little sample of the big csv file so I can quick look at things in R with less suffering
    small_output_csv = os.path.join(folder, 'concatenated_smol.csv')
    total_rows = result_df.shape[0]
    n = int(total_rows * 0.05)
    smol_result_df = result_df.sample(n=n, replace = False)
    smol_result_df.to_csv(small_output_csv, index=False)

    print('Concatenated CSV saved as: ' + output_csv)


# Usage
#folder = '/Users/Confocal/Desktop/ciliabulk2/processed/'
#mapping_csv = '/Users/Confocal/Desktop/ciliabulk2/cilia_ptskey.csv'
#timepoints_csv = '/Users/Confocal/Desktop/ciliabulk2/timepoints.csv'

#folder = 'D:/ciliako44/ciliavids/denoised/processed/'
#mapping_csv = 'D:/ciliako44/ciliako44_pts.csv'


#concatenate_csvs(folder, mapping_csv)

In [ ]:
# Usage: calculating beat frequency & making .csvs and image nuggets.
folder = 'D:/longcilia/denoised/unmatched/'
mk = perpair_processing(folder)

In [ ]:
# Usage: matching up the .csvs to relevant experimental info such as donor, condition, mock or infected, etc.
folder = 'D:/ciliakos_all/denoised/processed/ciliako44/'
mapping_csv = 'D:/ciliakos_all/denoised/processed/ciliako44_pts.csv'
timepoints_csv = 'D:/ciliakos_all/denoised/processed/ciliako44_timepoints.csv'

concatenate_csvs(folder, mapping_csv, timepoints_csv)